# Ejercicio 1: *Part Of Speech* usando un Perceptrón Estructurado

En este primer ejercicio se pide construir un perceptron estructurado y entrenarlo para hacer una predicción de *Part Of Speech* usando el dataset [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). Es un dataset que contiene secuencias de frases en ingles extraídas de libros y la entidad de cada palabra (nombre, vervo, determinante, etc.) correspondiente al POS.


## Enunciado:

#### 1. Entrenar un perpectrón estructurado para predecir Part Of Speech usando el dataset ConLL.

Además, responder a las siguientes preguntas:

* 1.1. ¿Cuantos features tiene el feature mapper? ¿Qué representan?
* 1.2. En una seucencia de entrenamiento, ¿cuandos tipos de features encontramos en una secuencia? ¿Qué nos indican?
* 1.3. Cuando construimos el SP, ¿cuantos estados posibles tiene y porqué?
* 1.4. Cuando construimos el SP, ¿cuantos parámetros tiene y porqué?


#### 2. Comparar los resultados con el HMM entrenado con el mismo dataset usado en la sesión 2 en clase.


#### 3. Comprovar si el perceptrón estructurado clasifica correctamente una palabra que no ha visto en el entrenamiento.





**Part Of Speech (POS)**

- Part Of Speech (POS) se refiere a la categoría gramatical o función sintáctica que desempeña una palabra en una frase. 
- Es un concepto lingüístico utilizado para clasificar las palabras en función de sus propiedades sintácticas y morfológicas. 
- El etiquetado POS consiste en asignar una etiqueta específica a cada palabra de una frase.


## Libraries 

In [14]:
import os, sys, inspect

import numpy as np

import skseq
import skseq.sequences
from skseq.sequences import sequence
import skseq.readers
import skseq.readers.pos_corpus



# To import modules or packages that are located in a directory above the current script's directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [16]:
# to read and handle part-of-speech tagging datasets
corpus = skseq.readers.pos_corpus.PostagCorpus()

# path to the directory where the CoNLL-2003 dataset is stored
data_path = "conll"

# train_seq: a list of sequences where each sequence represents a sentence in the training data, 
            # and each word in the sentence is paired with its corresponding part-of-speech tag.
    # read_sequence_list_conll method: to read the training data from the CoNLL-2003 dataset
    # max_sent_len: An optional argument that specifies the maximum sentence length to consider. In this case, it is set to 100.
    # max_nr_sent: An optional argument that specifies the maximum number of sentences to read. Here, it is set to 5000.
train_seq = corpus.read_sequence_list_conll(data_path + "/train-02-21.conll", max_sent_len=100, max_nr_sent=5000)

test_seq = corpus.read_sequence_list_conll(data_path + "/test-23.conll", max_sent_len=100, max_nr_sent=1000)

# dev_seq: a list of sequences representing the development sentences along with their corresponding part-of-speech tags.
dev_seq = corpus.read_sequence_list_conll(data_path + "/dev-22.conll", max_sent_len=100, max_nr_sent=1000)

In [18]:
train_seq[0]

0/0 1/1 2/2 3/3 4/2 5/0 6/4 7/1 8/2 9/4 10/0 11/2 12/5 13/2 14/2 15/4 6/4 16/6 17/2 18/6 19/1 20/2 21/0 22/2 23/2 24/4 9/4 25/2 26/7 27/2 28/4 24/4 19/1 29/2 5/0 30/2 24/4 31/6 32/0 33/2 34/2 24/4 35/6 36/8 37/6 38/5 39/2 40/2 41/4 

- Primera secuencia de los datos de entrenamiento.
- Cada token de la secuencia se representa como 'word_index/tag_index', donde 'word_index' es el índice de la palabra en la frase y 'tag_index' es el índice de la part-of-speech tag correspondiente a esa palabra. 
- Por ejemplo, '0/0' indica que la primera palabra de la frase tiene asignada la part-of-speech tag con índice 0.


In [21]:
train_seq.y_dict

{'adp': 0,
 'det': 1,
 'noun': 2,
 'num': 3,
 '.': 4,
 'prt': 5,
 'verb': 6,
 'conj': 7,
 'adv': 8,
 'pron': 9,
 'adj': 10,
 'x': 11}

El diccionario 'train_seq.y_dict' representa el diccionario que asigna part-of-speech tags a sus índices numéricos correspondientes. A cada part-of-speech tag se le asigna un índice único con fines de representación y cálculo en el modelo.

En el diccionario proporcionado tenemos lo siguiente: 

- 'adp': 0 representa la part-of-speech tag "adp" (preposición).
- 'det': 1 representa la part-of-speech tag "det" (determinante).
- 'noun': 2 representa la part-of-speech tag "noun" (sustantivo).
- 'num': 3 representa la parte de la etiqueta "num" (numeral).
- '.': 4 representa la part-of-speech tag '.' (signo de puntuación).
- 'prt': 5 representa la part-of-speech tag "prt" (partícula).
- 'verb': 6 representa la part-of-speech tag "verb" (verbo).
- 'conj': 7 representa la part-of-speech tag "conj" (conjunción).
- 'adv': 8 representa la part-of-speech tag "adv" (adverbio).
- 'pron': 9 representa la part-of-speech tag "pron" (pronombre).
- 'adj': 10 representa la part-of-speech tag "adj" (adjetivo).
- 'x': 11 representa la part-of-speech tag "x" (otro).

Este diccionario permite una correspondencia eficaz entre las etiquetas y sus representaciones numéricas durante el entrenamiento y la evaluación del modelo.

In [23]:
feature_mapper = skseq.sequences.id_feature.IDFeatures(train_seq)
feature_mapper